## Testing API batch in GPT

In [2]:
#!pip install openai==0.28

In [3]:
import pandas as pd
import openai

### Blood data:

In [4]:
df = pd.read_excel("data.xlsx")
df.head()

,ID_PESSOA,DT_NASCIMENTO,IDADE,NOME_EXAME,NOME_ITEM_DE_EXAME,VALOR_ITEM_DE_EXAME,UNIDADE_MEDIA,DATA_EXAME
0,31D9A8F2827B40DD6FB1AD78EB977FBD,1959-10-17,65,HEMOGRAMA,ERITROCITOS,3.35,milhoes/mm3,2024-01-19
1,31D9A8F2827B40DD6FB1AD78EB977FBD,1959-10-17,65,HEMOGRAMA,HEMOGLOBINA,11.20,g/dL,2024-01-19
2,31D9A8F2827B40DD6FB1AD78EB977FBD,1959-10-17,65,HEMOGRAMA,HEMATOCRITO,31.00,%,2024-01-19
3,31D9A8F2827B40DD6FB1AD78EB977FBD,1959-10-17,65,HEMOGRAMA,VCM,94.00,fL,2024-01-19
4,31D9A8F2827B40DD6FB1AD78EB977FBD,1959-10-17,65,HEMOGRAMA,HCM,33.00,pg,2024-01-19


In [5]:
len(df['ID_PESSOA'].unique())

1955

In [6]:
# Usando uma lista para armazenar IDs únicos conforme eles aparecem
primeiros_ids = []
for id_pessoa in df['ID_PESSOA']:
    if id_pessoa not in primeiros_ids:
        primeiros_ids.append(id_pessoa)
    if len(primeiros_ids) == 5:
        break

# Exibindo o resultado
print(primeiros_ids)

['31D9A8F2827B40DD6FB1AD78EB977FBD', '012328DB60F3D8E3958C89DB8039C5DD', '0865C0122500EE5FBBF21907613A11D4', 'EE9CC63706C583C739CB0653BEF9AA28', '6E3B796A05C4CB4A003B6612EBBC893F']


In [7]:
#filtrar o df pra pegar so 5 ids pra testar
df_5 = df[df['ID_PESSOA'].isin(primeiros_ids)]
len(df_5['ID_PESSOA'].unique())

5

In [8]:
#df_5.to_csv('df_5.csv', index=False)

In [9]:
# Agrupar os dados por paciente (ID_PESSOA)
grouped = df_5.groupby("ID_PESSOA")

# Transformar cada grupo em um dicionário
batch_data = {patient_id: group.to_dict(orient='records') for patient_id, group in grouped}

In [10]:
first_key, first_dict = next(iter(batch_data.items()))
print(first_key, first_dict)

012328DB60F3D8E3958C89DB8039C5DD [{'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 'DT_NASCIMENTO': Timestamp('1959-12-04 00:00:00'), 'IDADE': 65, 'NOME_EXAME': 'HEMOGRAMA', 'NOME_ITEM_DE_EXAME': 'ERITROCITOS', 'VALOR_ITEM_DE_EXAME': 4.37, 'UNIDADE_MEDIA': 'milhoes/mm3', 'DATA_EXAME': Timestamp('2024-01-19 00:00:00')}, {'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 'DT_NASCIMENTO': Timestamp('1959-12-04 00:00:00'), 'IDADE': 65, 'NOME_EXAME': 'HEMOGRAMA', 'NOME_ITEM_DE_EXAME': 'HEMOGLOBINA', 'VALOR_ITEM_DE_EXAME': 11.5, 'UNIDADE_MEDIA': 'g/dL', 'DATA_EXAME': Timestamp('2024-01-19 00:00:00')}, {'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 'DT_NASCIMENTO': Timestamp('1959-12-04 00:00:00'), 'IDADE': 65, 'NOME_EXAME': 'HEMOGRAMA', 'NOME_ITEM_DE_EXAME': 'HEMATOCRITO', 'VALOR_ITEM_DE_EXAME': 34.0, 'UNIDADE_MEDIA': '%', 'DATA_EXAME': Timestamp('2024-01-19 00:00:00')}, {'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 'DT_NASCIMENTO': Timestamp('1959-12-04 00:00:00'), 'IDADE': 65, 'NOME

In [10]:
prompt1 = """
Você tem o seguinte template de um dataframe de hemograma:

ID_PESSOA | DATA_EXAME | NOME_EXAME | NOME_ITEM_DE_EXAME | VALOR_ITEM_DE_EXAME | UNIDADE_DE_MEDIDA | VALOR_DE_REFERENCIA | LABORATORIO | DT_NASCIMENTO | DT_ULTIMA_MAMOGRAFIA

Exemplo:
A1 | 01/02/2023 | HEMOGRAMA | RDW | 15,10 | % | 11,8 - 14,2 % | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | LEUCOCITOS | 6.260,00 | células/mm³ | 3.470 - 8.290 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | PLAQUETAS | 300.500,00 | células/mm³ | 163.000 - 343.000 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020

Será feito o upload de arquivos contendo dados de hemogramas estruturados como no template acima. Extraia as informações e crie um dicionário `X` contendo 9 chaves, onde cada chave corresponde a uma das colunas do dataframe. Estruture as chaves e valores em snake_case, sem acentos (ex: "basofilos" ao invés de "basófilos"). O dicionário deve conter:

1. Na coluna 'valor_item_de_exame', ajuste:
   - Multiplique por 10³ os valores de plaquetas que estejam na casa das centenas (ex: 100 a 999).
   - Se 'segmentados' e 'bastonetes' forem fornecidos, some-os para obter o valor de neutrófilos. Exclua linhas de valores relativos após calcular os absolutos.
   - Para valores relativos em %, converta em absoluto para analitos específicos (ex: linfócitos, eosinófilos, basófilos, etc.) usando: `int(valor * leucocitos / 100)`.
   - A soma dos leucócitos deve ser coerente.
   

Precisa ter todos esses analitos, caso apareça no exame.
Me retorne APENAS o dicionário corrigido, sem explicações. Exemplo de resposta:

```python
x = {
    'ID_PESSOA': '0865C0122500EE5FBBF21907613A11D4',
    'DATA_EXAME': '2024-01-22',
    'NOME_EXAME': 'HEMOGRAMA',
    'NOME_ITEM_DE_EXAME': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS'],
    'VALOR_ITEM_DE_EXAME': [4.76, 13.6, 41.0, 86.0, 29.0, 33.0, 14.0, 14670.0, 0.0],
    'UNIDADE_DE_MEDIDA': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%'],
    'VALOR_DE_REFERENCIA': [],  
    'LABORATORIO': 'Lab1',
    'DT_NASCIMENTO': '1955-04-05',
    'DT_ULTIMA_MAMOGRAFIA': '2024-04-05', 
}
```"""

In [31]:
prompt2 = """
Você tem o seguinte template de um dataframe de hemograma:

ID_PESSOA | DATA_EXAME | NOME_EXAME | NOME_ITEM_DE_EXAME | VALOR_ITEM_DE_EXAME | UNIDADE_DE_MEDIDA | VALOR_DE_REFERENCIA | LABORATORIO | DT_NASCIMENTO | DT_ULTIMA_MAMOGRAFIA

Exemplo:
A1 | 01/02/2023 | HEMOGRAMA | RDW | 15,10 | % | 11,8 - 14,2 % | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | LEUCOCITOS | 6.260,00 | células/mm³ | 3.470 - 8.290 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | PLAQUETAS | 300.500,00 | células/mm³ | 163.000 - 343.000 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020

E mais linhas pro mesmo ID_PESSOA pois cada linha é um analito do hemograma.

Será feito o upload de arquivos contendo dados de hemogramas estruturados como no template acima. Extraia as informações e crie um dicionário `X` contendo 9 chaves, onde cada chave corresponde a uma das colunas do dataframe. Estruture as chaves e valores em snake_case, sem acentos (ex: "basofilos" ao invés de "basófilos"). O dicionário deve conter:

1. Na coluna 'valor_item_de_exame', ajuste:
   - Multiplique por 10³ os valores de plaquetas que estejam na casa das centenas (ex: 100 a 999) e substitua a unidade de medida por /mm³
 
Precisa ter todos esses analitos, caso apareça no exame.
Me retorne APENAS o dicionário corrigido, sem explicações. Exemplo de resposta:

```python
x = {
    'ID_PESSOA': '',
    'DATA_EXAME': '',
    'NOME_EXAME': '',
    'NOME_ITEM_DE_EXAME': [],
    'VALOR_ITEM_DE_EXAME': [],
    'UNIDADE_DE_MEDIDA': [],
    'VALOR_DE_REFERENCIA': [],  
    'LABORATORIO': '',
    'DT_NASCIMENTO': '',
    'DT_ULTIMA_MAMOGRAFIA': '', 
}
```
Caso nao tenha esses campos, deixe em branco. Unifique as unidades de medida para todas as pacientes.
"""

In [32]:
prompt3 = """
Você tem o seguinte template de um dataframe de hemograma:

ID_PESSOA | DATA_EXAME | NOME_EXAME | NOME_ITEM_DE_EXAME | VALOR_ITEM_DE_EXAME | UNIDADE_DE_MEDIDA | VALOR_DE_REFERENCIA | LABORATORIO | DT_NASCIMENTO | DT_ULTIMA_MAMOGRAFIA

Exemplo:
A1 | 01/02/2023 | HEMOGRAMA | RDW | 15,10 | % | 11,8 - 14,2 % | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | LEUCOCITOS | 6.260,00 | células/mm³ | 3.470 - 8.290 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | PLAQUETAS | 300.500,00 | células/mm³ | 163.000 - 343.000 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020

E mais linhas pro mesmo ID_PESSOA pois cada linha é um analito do hemograma.

Será feito o upload de arquivos contendo dados de hemogramas estruturados como no template acima. Extraia as informações e crie um dicionário `X` contendo 9 chaves, onde cada chave corresponde a uma das colunas do dataframe. Estruture as chaves e valores em snake_case, sem acentos (ex: "basofilos" ao invés de "basófilos"). O dicionário deve conter:

1. Na coluna 'valor_item_de_exame', ajuste:
   - Para valores relativos em % de LINFOCITOS, SEGMENTADOS, MONOCITOS, BASTONETES, EOSINOFILOS, BASOFILOS converta em absoluto para analitos específicos usando o calculo: `int(valor * leucocitos / 100)` e substitua a unidade de medida por /mm³
   
Precisa ter todos esses analitos, caso apareça no exame.
Me retorne APENAS o dicionário corrigido, sem explicações. Exemplo de resposta:

```python
x = {
    'ID_PESSOA': '',
    'DATA_EXAME': '',
    'NOME_EXAME': '',
    'NOME_ITEM_DE_EXAME': [],
    'VALOR_ITEM_DE_EXAME': [],
    'UNIDADE_DE_MEDIDA': [],
    'VALOR_DE_REFERENCIA': [],  
    'LABORATORIO': '',
    'DT_NASCIMENTO': '',
    'DT_ULTIMA_MAMOGRAFIA': '', 
}
```
Caso nao tenha esses campos, deixe em branco. Unifique as unidades de medida, datas e nomes de analitos para todas as pacientes.
"""

In [57]:
prompt = """
Você tem o seguinte template de um dataframe de hemograma:

ID_PESSOA | DATA_EXAME | NOME_EXAME | NOME_ITEM_DE_EXAME | VALOR_ITEM_DE_EXAME | UNIDADE_DE_MEDIDA | VALOR_DE_REFERENCIA | LABORATORIO | DT_NASCIMENTO | DT_ULTIMA_MAMOGRAFIA

Exemplo:
A1 | 01/02/2023 | HEMOGRAMA | RDW | 15,10 | % | 11,8 - 14,2 % | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | LEUCOCITOS | 6.260,00 | células/mm³ | 3.470 - 8.290 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020
A1 | 01/02/2023 | HEMOGRAMA | PLAQUETAS | 300.500,00 | células/mm³ | 163.000 - 343.000 células/mm³ | Lab01 | 01/01/1963 | 01/02/2020

Será feito o upload de arquivos contendo dados de hemogramas estruturados como no template acima. Extraia as informações e crie um dicionário `X` contendo 9 chaves, onde cada chave corresponde a uma das colunas do dataframe. Estruture as chaves e valores em snake_case, sem acentos (ex: "basofilos" ao invés de "basófilos"). O dicionário deve conter:

1. `ID_PESSOA`: ID da paciente
2. `DATA_EXAME`: Data do exame (formato data)
3. `NOME_EXAME`: Sempre "HEMOGRAMA" (string em caixa alta). Se outro nome aparecer e 'nome_item_de_exame' não for um analito, excluir a linha.
4. `NOME_ITEM_DE_EXAME`: Nome do analito (string em caixa alta)
5. `VALOR_ITEM_DE_EXAME`: Valor do analito (float)
6. `UNIDADE_DE_MEDIDA`: Unidade de medida (string)
7. `VALOR_DE_REFERENCIA`: Valor de referência do analito (string)
8. `LABORATORIO`: nome do laboratorio (string)
9. `DT_NASCIMENTO`: Data de nascimento da paciente (formato data)
10. `DT_ULTIMA_MAMOGRAFIA`: Data da última mamografia (formato data)

Considere os seguintes ajustes e cálculos:

1. Na coluna 'valor_item_de_exame', ajuste:
   - Multiplique por 10³ os valores de plaquetas que estejam na casa das centenas (ex: 100 a 999).
   - Se 'segmentados' e 'bastonetes' forem fornecidos, some-os para obter o valor de neutrófilos. Exclua linhas de valores relativos após calcular os absolutos.
   - Para valores relativos em %, converta em absoluto para analitos específicos (ex: linfócitos, eosinófilos, basófilos, etc.) usando: `int(valor * leucocitos / 100)`.
   - A soma dos leucócitos deve ser coerente.
   
2. Corrija as unidades de medida se houver inconsistências entre o valor e a unidade (ex: % ao invés de /mm³).

Analitos permitidos e suas unidades:
- RDW (%)
- LEUCOCITOS (células/mm³)
- LINFOCITOS (células/mm³)
- PLAQUETAS (células/mm³)
- ERITROCITOS (milhões/µL)
- HEMOGLOBINA (g/dL)
- BLASTOS (%)
- MONOCITOS (células/mm³)
- EOSINOFILOS (células/mm³)
- NEUTROFILOS (células/mm³)

Precisa ter todos esses analitos, caso apareça no exame.
Me retorne APENAS o dicionário corrigido, sem explicações. Exemplo de resposta:

```python
x = {
    'ID_PESSOA': '0865C0122500EE5FBBF21907613A11D4',
    'DATA_EXAME': '2024-01-22',
    'NOME_EXAME': 'HEMOGRAMA',
    'NOME_ITEM_DE_EXAME': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS'],
    'VALOR_ITEM_DE_EXAME': [4.76, 13.6, 41.0, 86.0, 29.0, 33.0, 14.0, 14670.0, 0.0],
    'UNIDADE_DE_MEDIDA': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%'],
    'VALOR_DE_REFERENCIA': [],  
    'LABORATORIO': 'Lab1',
    'DT_NASCIMENTO': '1955-04-05',
    'DT_ULTIMA_MAMOGRAFIA': '2024-04-05', 
}
```"""

In [19]:
# Defina sua chave de API
openai.api_key = 'your_api_key_here'
openai.organization = 'your_organization_key_here'

In [20]:
def enviar_batch(batch, prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # ou "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "Você é um analista e engenheiro expert em data science, dados médicos de exames e engenehiro de dados."},
            {"role": "user", "content": prompt},
            {"role": "user", "content": batch}
        ],
        max_tokens=1500,  # Ajuste conforme necessário
        n=1,
        temperature=0.7
    )
    return response

In [29]:
df[df['ID_PESSOA'] == '012328DB60F3D8E3958C89DB8039C5DD']

,ID_PESSOA,DT_NASCIMENTO,IDADE,NOME_EXAME,NOME_ITEM_DE_EXAME,VALOR_ITEM_DE_EXAME,UNIDADE_MEDIA,DATA_EXAME
20,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,ERITROCITOS,4.37,milhoes/mm3,2024-01-19
21,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,HEMOGLOBINA,11.50,g/dL,2024-01-19
22,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,HEMATOCRITO,34.00,%,2024-01-19
23,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,VCM,79.00,fL,2024-01-19
24,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,HCM,26.00,pg,2024-01-19
25,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,CHBCM,34.00,%,2024-01-19
26,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,RDW,16.00,%,2024-01-19
27,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,LEUCOCITOS,8190.00,/mm3,2024-01-19
28,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,PROMIELOCITOS,0.00,%,2024-01-19
29,012328DB60F3D8E3958C89DB8039C5DD,1959-12-04,65,HEMOGRAMA,MIELOCITOS,0.00,%,2024-01-19


In [33]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams]) 
    resposta = enviar_batch(batch_str, prompt3)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 x = {
    'id_pessoa': '012328DB60F3D8E3958C89DB8039C5DD',
    'data_exame': '2024-01-19',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS', 'MIELOCITOS', 'METAMIELOCITOS', 'BASTONETES', 'SEGMENTADOS', 'EOSINOFILOS', 'BASOFILOS', 'LINFOCITOS', 'LINFOCITOS ATIPICOS', 'MONOCITOS', 'BLASTOS', 'PLAQUETAS'],
    'valor_item_de_exame': [4.37, 11.5, 34.0, 79.0, 26.0, 34.0, 16.0, 8190.0, 0.0, 0.0, 0.0, 1.0, 72.0, 2.0, 0.0, 16.0, 0.0, 9.0, 0.0, 158.0],
    'unidade_de_medida': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', 'mil/mm3'],
    'valor_de_referencia': [],
    'laboratorio': '',
    'dt_nascimento': '1959-12-04',
    'dt_ultima_mamografia': '',
}
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 ```python
x = {
    'id_pessoa': '0865C0122500EE5FBBF2

In [26]:
#funcionou para plaquetas
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt2)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 ```python
x = {
    'id_pessoa': '012328DB60F3D8E3958C89DB8039C5DD',
    'data_exame': '2024-01-19',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS', 'MIELOCITOS', 'METAMIELOCITOS', 'BASTONETES', 'SEGMENTADOS', 'EOSINOFILOS', 'BASOFILOS', 'LINFOCITOS', 'LINFOCITOS ATIPICOS', 'MONOCITOS', 'BLASTOS', 'PLAQUETAS'],
    'valor_item_de_exame': [4.37, 11.5, 34.0, 79.0, 26.0, 34.0, 16.0, 8190.0, 0.0, 0.0, 0.0, 1.0, 72.0, 2.0, 0.0, 16.0, 0.0, 9.0, 0.0, 177000.0],
    'unidade_de_medida': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '/mm3'],
    'valor_de_referencia': [],
    'laboratorio': '',
    'dt_nascimento': '1959-12-04',
    'dt_ultima_mamografia': '', 
}
```
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 ```python
x = {
    'id_pessoa': '0865C

In [23]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt2)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 ```python
x = {
    'id_pessoa': 'A1',
    'data_exame': '01/02/2023',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['RDW', 'LEUCOCITOS', 'PLAQUETAS'],
    'valor_item_de_exame': [15.10, 6260.00, 300500.00],
    'unidade_de_medida': ['%', 'células/mm³', 'células/mm³'],
    'valor_de_referencia': ['11,8 - 14,2 %', '3.470 - 8.290 células/mm³', '163.000 - 343.000 células/mm³'],
    'laboratorio': 'Lab01',
    'dt_nascimento': '01/01/1963',
    'dt_ultima_mamografia': '01/02/2020', 
}
```
Essa é a resposta corrigida. O valor de plaquetas foi multiplicado por 10³ e a unidade de medida foi alterada para células/mm³.
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 ```python
x = {
    'id_pessoa': '0865C0122500EE5FBBF21907613A11D4',
    'data_exame': '2024-01-22',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCIT

In [21]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt2)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 ```python
x = {
    'id_pessoa': '012328DB60F3D8E3958C89DB8039C5DD',
    'data_exame': '2024-01-19',
    'nome_exame': 'hemograma',
    'nome_item_de_exame': ['eritrocitos', 'hemoglobina', 'hematocrito', 'vcm', 'hcm', 'chbcm', 'rdw', 'leucocitos', 'promielocitos', 'mielocitos', 'metamielocitos', 'bastonetes', 'segmentados', 'eosinofilos', 'basofilos', 'linfocitos', 'linfocitos_atipicos', 'monocitos', 'blastos', 'plaquetas'],
    'valor_item_de_exame': [4.37, 11.5, 34.0, 79.0, 26.0, 34.0, 16.0, 8190.0, 0.0, 0.0, 0.0, 1.0, 72.0, 2.0, 0.0, 16.0, 0.0, 9.0, 0.0, 158000.0],
    'unidade_de_medida': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', 'mil/mm3'],
    'valor_de_referencia': [],
    'laboratorio': '',
    'dt_nascimento': '1959-12-04',
    'dt_ultima_mamografia': '', 
}
```

Nota: Ajustei o valor das plaquetas de 158.0 para 158000.0 como especificado na pergunta. 

In [16]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt2)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 ```python
x = {
    'id_pessoa': '012328DB60F3D8E3958C89DB8039C5DD',
    'data_exame': '2024-01-19',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS', 'MIELOCITOS', 'METAMIELOCITOS', 'BASTONETES', 'SEGMENTADOS', 'EOSINOFILOS', 'BASOFILOS', 'LINFOCITOS', 'LINFOCITOS ATIPICOS', 'MONOCITOS', 'BLASTOS', 'PLAQUETAS'],
    'valor_item_de_exame': [4.37, 11.5, 34.0, 79.0, 26.0, 34.0, 16.0, 8190.0, 0.0, 0.0, 0.0, 1.0, 72.0, 2.0, 0.0, 16.0, 0.0, 9.0, 0.0, 158000.0],
    'unidade_de_medida': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', '/mm3', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', 'mil/mm3'],
    'valor_de_referencia': [],
    'laboratorio': 'Lab1',
    'dt_nascimento': '1959-12-04',
    'dt_ultima_mamografia': '2024-04-05', 
}
```
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 x = {
    'id_pessoa':

In [78]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 {
    'id_pessoa': 'A1',
    'data_exame': '01/02/2023',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['RDW', 'LEUCOCITOS', 'PLAQUETAS'],
    'valor_item_de_exame': [15.1, 6260, 300500],
    'unidade_de_medida': ['%', 'células/mm³', 'células/mm³'],
    'valor_de_referencia': ['11,8 - 14,2 %', '3.470 - 8.290 células/mm³', '163.000 - 343.000 células/mm³'],
    'laboratorio': 'Lab01',
    'dt_nascimento': '01/01/1963',
    'dt_ultima_mamografia': '01/02/2020'
}
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 {'id_pessoa': 'A1', 
 'data_exame': '01/02/2023', 
 'nome_exame': 'HEMOGRAMA', 
 'nome_item_de_exame': ['RDW', 'LEUCOCITOS', 'PLAQUETAS'], 
 'valor_item_de_exame': [15.10, 6260.00, 300500.00], 
 'unidade_de_medida': ['%', 'células/mm³', 'células/mm³'], 
 'valor_de_referencia': ['11,8 - 14,2 %', '3.470 - 8.290 células/mm³', '163.000 - 343.000 células/mm³'], 
 'laboratorio': 'Lab01', 
 'dt_nascimento': '01/01/1963

In [55]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 ```python
x = {
    'id_pessoa': '012328DB60F3D8E3958C89DB8039C5DD',
    'data_exame': '2024-01-19',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['ERITROCITOS', 'HEMOGLOBINA', 'HEMATOCRITO', 'VCM', 'HCM', 'CHBCM', 'RDW', 'LEUCOCITOS', 'PROMIELOCITOS', 'MIELOCITOS', 'METAMIELOCITOS', 'BASTONETES', 'SEGMENTADOS', 'EOSINOFILOS', 'BASOFILOS', 'LINFOCITOS', 'LINFOCITOS ATIPICOS', 'MONOCITOS', 'BLASTOS', 'PLAQUETAS'],
    'valor_item_de_exame': [4.37, 11.5, 34.0, 79.0, 26.0, 34.0, 16.0, 8190.0, 0.0, 0.0, 0.0, 1.0, 72.0, 2.0, 0.0, 16.0, 0.0, 9.0, 0.0, 158000.0],
    'unidade_de_medida': ['milhoes/mm3', 'g/dL', '%', 'fL', 'pg', '%', '%', 'celulas/mm3', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', '%', 'celulas/mm3'],
    'valor_de_referencia': [],
    'laboratorio': 'Lab01',
    'dt_nascimento': '1959-12-04',
    'dt_ultima_mamografia': None
}
```
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 ```python
x = {
    

In [48]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 {'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 
'DATA_EXAME': '2024-01-19', 
'NOME_EXAME': 'HEMOGRAMA',
'NOME_ITEM_DE_EXAME': ['ERITROCITOS', 'HEMOGLOBINA', 'VCM', 'HCM', 'RDW', 'LEUCOCITOS', 'SEGMENTADOS', 'EOSINOFILOS', 'LINFOCITOS', 'MONOCITOS', 'PLAQUETAS'],
'VALOR_ITEM_DE_EXAME': [4.37, 11.5, 79.0, 26.0, 16.0, 8190.0, 72.0, 2.0, 16.0, 9.0, 177000.0],
'UNIDADE_DE_MEDIDA': ['milhoes/mm3', 'g/dL', 'fL', 'pg', '%', '/mm3', '%', '%', '%', '%', 'celulas/mm3'],
'VALOR_DE_REFERENCIA': [],
'LABORATORIO': '',
'DT_NASCIMENTO': '1959-12-04',
'DT_ULTIMA_MAMOGRAFIA': ''
}
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 ```python
x = {
    'id_pessoa': 'A1',
    'data_exame': '01/02/2023',
    'nome_exame': 'HEMOGRAMA',
    'nome_item_de_exame': ['RDW', 'LEUCOCITOS', 'PLAQUETAS'],
    'valor_item_de_exame': [15.10, 6260.00, 300500.00],
    'unidade_de_medida': ['%', 'células/mm³', 'células/mm³'],
    'valor_de_referencia': ['11,8 -

In [12]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 Os dicionários extraídos, ajustados conforme as instruções dadas, são os seguintes:

```python
{
'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 
'DATA_EXAME': '2024-01-19', 
'NOME_EXAME': 'HEMOGRAMA', 
'NOME_ITEM_DE_EXAME': 'ERITROCITOS', 
'VALOR_ITEM_DE_EXAME': 4.37, 
'UNIDADE_DE_MEDIDA': 'milhoes/mm3', 
'VALOR_DE_REFERENCIA': '3.9 a 5.0 milhoes/mm3', 
'DT_NASCIMENTO': '1959-12-04', 
'DT_ULTIMA_MAMOGRAFIA': '2023-01-19'
}

{
'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 
'DATA_EXAME': '2024-01-19', 
'NOME_EXAME': 'HEMOGRAMA', 
'NOME_ITEM_DE_EXAME': 'LEUCOCITOS', 
'VALOR_ITEM_DE_EXAME': 8190.0, 
'UNIDADE_DE_MEDIDA': '/mm3', 
'VALOR_DE_REFERENCIA': '4500 a 11000 /mm3', 
'DT_NASCIMENTO': '1959-12-04', 
'DT_ULTIMA_MAMOGRAFIA': '2023-01-19'
}

{
'ID_PESSOA': '012328DB60F3D8E3958C89DB8039C5DD', 
'DATA_EXAME': '2024-01-19', 
'NOME_EXAME': 'HEMOGRAMA', 
'NOME_ITEM_DE_EXAME': 'PLAQUETAS', 
'VALOR_ITEM_DE_EXAME': 158000.0, 
'UNIDADE_DE

In [14]:
for patient_id, exams in batch_data.items():
    batch_str = "\n".join([str(exam) for exam in exams])  # Formatar o batch como string
    resposta = enviar_batch(batch_str, prompt)
    print(f"Paciente {patient_id} - Resposta:\n", resposta['choices'][0]['message']['content'])

Paciente 012328DB60F3D8E3958C89DB8039C5DD - Resposta:
 O assistente não conseguiu gerar um dicionário a partir das informações fornecidas. Por favor, forneça as informações completas e corretas para que o assistente possa processá-las corretamente.
Paciente 0865C0122500EE5FBBF21907613A11D4 - Resposta:
 {'id_pessoa': '0865C0122500EE5FBBF21907613A11D4', 'data_exame': '2024-01-22', 'nome_exame': 'HEMOGRAMA', 'nome_item_de_exame': 'ERITROCITOS', 'valor_item_de_exame': 4.76, 'unidade_de_medida': 'milhoes/mm3', 'valor_de_referencia': '4.5-5.5 milhões/mm3', 'dt_nascimento': '1955-04-05', 'dt_ultima_mamografia': '2023-01-22'} 
{'id_pessoa': '0865C0122500EE5FBBF21907613A11D4', 'data_exame': '2024-01-22', 'nome_exame': 'HEMOGRAMA', 'nome_item_de_exame': 'HEMOGLOBINA', 'valor_item_de_exame': 13.6, 'unidade_de_medida': 'g/dL', 'valor_de_referencia': '12-16 g/dL', 'dt_nascimento': '1955-04-05', 'dt_ultima_mamografia': '2023-01-22'}
{'id_pessoa': '0865C0122500EE5FBBF21907613A11D4', 'data_exame': '20

['31D9A8F2827B40DD6FB1AD78EB977FBD', '012328DB60F3D8E3958C89DB8039C5DD', '0865C0122500EE5FBBF21907613A11D4', 'EE9CC63706C583C739CB0653BEF9AA28', '6E3B796A05C4CB4A003B6612EBBC893F', 'E9B78DFE5A398D73C587DE4396E5D961', '0ABA63FFB81F033F451DCD6F0C5C5054', 'FD25B5A8C91DF8DD5F9995B7B81A6CBD', 'FFA0CB9EAE21B940DF78B82EAAA8BAF3', '784D604D2139352E978180FA4916BB5A']